## 模型与剖分

In [1]:
import numpy as np
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

from fealpy.decorator import cartesian
from fealpy.mesh import TriangleMesh
from fealpy.functionspace import LagrangeFiniteElementSpace
from fealpy.boundarycondition import DirichletBC

class pde():
    def __init__(self, mu=1, lam=1):
        self.mu  = mu
        self.lam = lam
        
    def domain(self):
        return [0, 1, 0, 1]
    
    def init_mesh(self, n=1, meshtype='tri'):
        node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
        cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
        mesh = TriangleMesh(node, cell)
        mesh.uniform_refine(n)
        return mesh
    
    @cartesian
    def source(self, p):
        x   = p[..., 0]
        y   = p[..., 1]
        mu  = self.mu
        lam = self.lam
        
        sin = np.sin
        cos = np.cos
        val = np.zeros(p.shape, dtype=np.float64)
        
        val[..., 0] = -((2 * mu + lam) * y * (y - 1) * (2 * cos(x) - (x - 1) * sin(x))
                        + (mu + lam) * (2 * x - 1) * (sin(y) + (y - 1) * cos(y)) 
                        + 2 * mu * (x -1) * sin(x))
        val[..., 1] = -((2 * mu + lam) * x * (x - 1) * (2 * cos(y) - (y - 1) * sin(y))
                        + (mu + lam) * (2 * y - 1) * (sin(x) + (x - 1) * cos(x))
                        + 2 * mu * (y - 1) * sin(y))

        return val
    
    def dirichlet(self, p):
        var = np.zeros_like(p)
        return var
    
    def solution(self, p):
        x = p[..., 0]
        y = p[..., 1]
        
        val = np.zeros(p.shape, dtype=np.float64)
        
        val[..., 0] = y * (x - 1) * (y - 1) * np.sin(x)
        val[..., 1] = x * (x - 1) * (y - 1) * np.sin(y)
        return val
        
    def is_dirichlet_boundary(self, p):
        x = p[..., 0]
        y = p[..., 1]
        flag = np.max(np.abs(y)) < 1e-13 
        return flag
    
def error(u, uh):
    e = u - uh
    emax = np.max(np.abs(e))
    return emax
    
pde = pde()
mesh = pde.init_mesh(1)
NN = mesh.number_of_nodes()
NC = mesh.number_of_cells()
#[NC,2] 剖分点及其编号(下标)
node = mesh.entity('node') 
#[NC,3] 剖分区间及其端点编号
cell = mesh.entity('cell') 
#[NC] 每个单元的面积
cm = mesh.entity_measure()

##[NC,3,2] 每个单元上基函数(三个)对x、y的偏导数
#glam = mesh.grad_lambda()
#GLAM = np.broadcast_to(glam[:, :, None, :], shape=[NC,3,2,2])
#GLAM_T = GLAM.swapaxes(2,3)
#GLAM = GLAM + GLAM_T
#
##[NC,3,3] 单元刚度矩阵第一部分
#S = np.einsum("cnij, cmij, c->cnm", GLAM, GLAM, cm)
#glam_sum = np.einsum("cni->cn", glam)
##[NC,3,3] 单元刚度矩阵第二部分
#M = np.einsum("cn, cm->cnm", glam_sum, glam_sum)
#
#I = np.broadcast_to(cell[:, :, None], shape=S.shape)
#J = np.broadcast_to(cell[:, None, :], shape=S.shape)
#
#S = csr_matrix((S.flat, (I.flat, J.flat)), shape=(NN,NN))
#M = csr_matrix((M.flat, (I.flat, J.flat)), shape=(NN,NN))
#A = pde.mu * S / 2 + pde.lam * M

## 刚度矩阵

每个单元六个基函数，为$\varphi_1(第一个节点x方向), \varphi_2(第一个节点y方向), \varphi_3(第二个节点x方向), \varphi_4(第二个节点y方向), \varphi_5(第三个节点x方向), \varphi_6(第三个节点y方向)$ 

In [2]:
##[NC,3,2] 每个单元上基函数(三个)对x、y的偏导数
glam_x_y = mesh.grad_lambda()
#print("glam_x_y= ", glam_x_y)
glam_dim4 = np.broadcast_to(glam_x_y[:, :, None, :], shape=(NC,3,2,2)).copy()
#print("glam_dim4= ", glam_dim4)
glam_dim5 = np.broadcast_to(glam_dim4[:, :, :, None, :], shape=(NC,3,2,2,2)).copy()
#print("glam_dim5= ", glam_dim5)
glam_dim5[:, :, 0, 1, :] = 0 # [NC,3(三个节点),2(每个点两个基函数),2,2(后两个指标指基函数的 grad)]
glam_dim5[:, :, 1, 0, :] = 0 # x方向上的基函数第二个分量为0，y方向上的基函数第一个分量为0
#print("glam_dim5= ", glam_dim5)

# [NC,3,2]
glam_sum = np.einsum("cnmij->cnm", glam_dim5)
# [NC,6(每个基函数的 div)]
glam_sum = glam_sum.reshape(NC,6)
#print("glam_sum= ", glam_sum)

glam_t = glam_dim5.swapaxes(3,4)
glam_dim5 = glam_dim5 + glam_t
#print("glam_dim5= ", glam_dim5)
glam_dim4 = glam_dim5.reshape(NC,6,2,2)  #后两个指标指每个基函数的 grad + grad^t
#print("glam_dim4= ", glam_dim4)

# [NC,6,6]
S = np.einsum("cnij, cmij, c->cnm", glam_dim4, glam_dim4, cm)
#[NC,6,6] 单元刚度矩阵第二部分
M = np.einsum("cn, cm, c->cnm", glam_sum, glam_sum, cm)

cell_x_y = np.broadcast_to(cell[:,:,None], shape=(NC, 3, 2)).copy()
cell_x_y[:,:,0] = 2 * cell_x_y[:,:,0]       #[NC,3] 三个节点x方向上基函数在总刚度矩阵的位置
cell_x_y[:,:,1] = 2 * cell_x_y[:,:,1] + 1   #[NC,3] 三个节点y方向上基函数在总刚度矩阵的位置
cell_x_y = cell_x_y.reshape(NC, 6)
I = np.broadcast_to(cell_x_y[:, :, None], shape=S.shape)
J = np.broadcast_to(cell_x_y[:, None, :], shape=S.shape)

S = csr_matrix((S.flat, (I.flat, J.flat)), shape=(2 * NN,2 * NN))
M = csr_matrix((M.flat, (I.flat, J.flat)), shape=(2 * NN,2 * NN))
A = pde.mu * S / 2 + pde.lam * M
#print("A= ", A.toarray())

In [3]:
def is_Bd_Node_flag():
    n = mesh.number_of_nodes()
    isBdNode = np.zeros(n, dtype=bool)
    for i in range(n):
        if (pde.is_dirichlet_boundary(mesh.node[i])):
            isBdNode[i] = True
    return isBdNode

## 载荷向量与求解

node[cell] [NC,3(三个节点),2(每个节点的 x, y 坐标)] 每个区间上节点的编号为 i,j,k

bb 编号与 $\varphi$ 相同

In [4]:
from fealpy.quadrature import TriangleQuadrature

NQ = 1
Q = int(NQ * (NQ + 1) / 2) # 内积分点数
qf = TriangleQuadrature(NQ)

#bcs [Q,3]; WS[Q,]
bcs,ws = qf.get_quadrature_points_and_weights()
#print("bcs= ", bcs)
#phi [Q,3,2]
phi = np.broadcast_to(bcs[:, :, None], shape=[Q,3,2])
#print("phi= ", phi)

#[Q,3,2(每个内积分节点两个基函数),2(每个基函数两个分量)]
phi_x_y = np.zeros((Q, 3, 2, 2), dtype=np.float64) 
phi_x_y[:, :, :, 0] = phi
phi_x_y[:, :, :, 1] = phi
phi_x_y[:, :, 0, 1] = 0
phi_x_y[:, :, 1, 0] = 0
#print("phi_x_y= ", phi_x_y)
phi_x_y = phi_x_y.reshape(Q,6,2)

# node[cell] [NC,3(三个节点),2(每个节点的 x, y 坐标)]
# ps [Q,NC,2]
# 内积分点标准单元坐标转换为真实坐标
ps = np.einsum('qi, cim->qcm', bcs, node[cell]) 
#print("ps= ", ps)
# val [Q,NC,2] 每个内积分的右端项的值
val = pde.source(ps)
#print("val= ", val)
#bb [NC,6]
bb = np.einsum('q, qcj, qij, c->ci', ws, val, phi_x_y, cm)
#print("cell= ", cell)
#print("bb= ", bb)
#print("cel_x_y= ", cell_x_y)

F = np.zeros(2 * NN)
np.add.at(F, cell_x_y, bb)
#print("F= ", F)

isBdNode = mesh.ds.boundary_node_flag()
isInterNode = ~isBdNode
#print("isInterNode= ", isInterNode)
isInterNode = np.broadcast_to(isInterNode[:, None], shape=(NN, 2))
isInterNode = isInterNode.reshape(2 * NN)
#print("isInterNode= ", isInterNode)

uh = np.zeros((2 * NN), dtype=np.float64)
uh[isInterNode] = spsolve(A[:, isInterNode][isInterNode], F[isInterNode])
#print("uh= ", uh)
uh = uh.reshape(NN, 2)
#print("uh= ", uh)

In [5]:
#def source(p):
#    val = np.ones_like(p)
#    return val
#
#val_exam = source(ps)
#print("val_exam= ", val_exam)
#quad = np.einsum('q, qcj, c->cj', ws, val_exam, cm)
#print("quad= ", quad)

In [6]:
#print("NQ= ", NQ)
#print("Q= ", Q)
#print("bcs= ", bcs)
#print("ws= ", ws)

In [7]:
u = pde.solution(node)
e = error(u, uh)
print("e= ", e)
print("u= ", u)
print("uh= ", uh)

e=  0.02511170908288423
u=  [[ 0.          0.        ]
 [-0.         -0.        ]
 [ 0.          0.        ]
 [-0.         -0.        ]
 [ 0.          0.        ]
 [ 0.05992819  0.05992819]
 [ 0.          0.        ]
 [-0.         -0.        ]
 [-0.         -0.        ]]
uh=  [[0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.03481648 0.03481648]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]]


In [8]:
#fig = plt.figure()
#ax = fig.gca()
#mesh.add_plot(ax)
#mesh.find_node(ax,showindex=True)
#mesh.find_cell(ax,showindex=True)